<a href="https://colab.research.google.com/github/MPrazzoli/AI_driven_investment_strategy/blob/main/LSTM_single_stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!python --version

Python 3.7.10


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import csv
from datetime import date, timedelta

# multivariate lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [ ]:
class CorrelationClass(object):
    # Initialization of the CorrelationClass object with the ticker symbol
    def __init__(self, ticker):
        self.principal_ticker = ticker
        self.correlated_tickers_for_open_prediction = None # we compute the correlation between open and all other stock close/adjclose price
        self.correlated_tickers_for_close_prediction = None # we compute the correlation between close/adjclose and all other stock open price

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the dataset
    if end_ix > len(sequences):
        break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

In [ ]:
open_df = pd.read_csv('/content/drive/MyDrive/_8_0_exported_dataframe/open', index_col='date')
adjclose_df = pd.read_csv('/content/drive/MyDrive/_8_0_exported_dataframe/adjclose', index_col='date')